# create metadata NOAA DSDP files
## 1-96 taxa, lithology, age, hard rocks

Get basic metadata (file names, column names) about NOAA DSDP dataset. Create csv that lists all the files.

NOAA_csv/DSDP_core_data  
expedition 1-96  
taxa, lithology, age models, hard rocks

In [1]:
import sys
sys.path.append('../../')
import glob
from pathlib import Path
import os

import pandas as pd
import numpy as np

from scripts.normalize_noaa_files import (
    unique_filenames_for_paths,
    unique_columns_for_paths,
    filename_index,
    format_filepaths_set
)
from config import OUTPUT_DIR, CLEAN_DATA_DIR

In [2]:
base_dir = CLEAN_DATA_DIR
data_dir = base_dir/'NOAA'/'DSDP_core_data'
metadata_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_dsdp_files.csv'

In [3]:
csv_paths = list(data_dir.glob("**/*.csv"))
print('files', len(csv_paths))

files 4479


## unique file names

Get all the file names.

In [4]:
unique_filenames_for_paths(csv_paths)

{'ageprof.csv',
 'b_forams.csv',
 'diatoms.csv',
 'dinoflag.csv',
 'ebri_act.csv',
 'hr_desc.csv',
 'nannos.csv',
 'ostracod.csv',
 'p_forams.csv',
 'phyliths.csv',
 'pollen.csv',
 'radiolar.csv',
 'siliflag.csv',
 'vistxt.csv'}

##  create file list metadata
Create csv that lists all the files for this dataset.

In [5]:
file_list = []
switch = {
    'b_forams.csv': 'benthic_foraminfera',
    'diatoms.csv': 'diatoms',
    'dinoflag.csv': 'dinoflagellates',
    'ebri_act.csv': 'ebridians',
    'nannos.csv': 'nannofossils',
    'ostracod.csv': 'ostracods',
    'phyliths.csv': 'phytoliths',
    'p_forams.csv': 'planktic_foraminfera',
    'pollen.csv': 'pollen',
    'radiolar.csv': 'radiolarians',
    'siliflag.csv': 'silicoflagellates',
}

for path in csv_paths:
    if '.ipynb_checkpoints' in str(path):
        continue
        
    file_data = {}
    relative_path = path.relative_to(base_dir)
    path_parts = relative_path.parts
    filename = relative_path.name
    
    file_data['path'] = relative_path
    
    if filename == 'vistxt.csv':
        type = 'lithology'
    elif filename == 'ageprof.csv':
        type = 'age'
    elif filename == 'hr_desc.csv':
        type = 'hard_rock'
    else:
        type = 'taxa'
    file_data['type'] = type
        
    if type == 'taxa':
        file_data['taxon_group'] = switch.get(filename, np.nan)
    
    file_data['expedition'] =  path_parts[2]
    file_data['site'] =  path_parts[3]

    file_list.append(file_data)

In [6]:
df = pd.DataFrame(file_list)
df.head()

,path,type,taxon_group,expedition,site
0,NOAA/DSDP_core_data/61/462/radiolar.csv,taxa,radiolarians,61,462
1,NOAA/DSDP_core_data/61/462/ageprof.csv,age,NaN,61,462
2,NOAA/DSDP_core_data/61/462/b_forams.csv,taxa,benthic_foraminfera,61,462
3,NOAA/DSDP_core_data/61/462/p_forams.csv,taxa,planktic_foraminfera,61,462
4,NOAA/DSDP_core_data/61/462/hr_desc.csv,hard_rock,NaN,61,462


In [7]:
df.to_csv(metadata_path, index=False)

## create grouped file list metadata

create list of files grouped by expedition and file type

In [8]:
contents = {}

for path in csv_paths:
    relative_path = path.relative_to(base_dir)
    parts = relative_path.parts
    filename = relative_path.name
    exp = parts[2]
     
    if exp not in contents:
        contents[exp] = {'taxa': set(), 'lithology': set(), 'age_model': set(), 'hard_rock': set()}
        
    if filename == 'vistxt.csv':
        contents[exp]['lithology'].add(filename)
    elif filename == 'ageprof.csv':
        contents[exp]['age_model'].add(filename)
    elif filename == 'hr_desc.csv':
        contents[exp]['hard_rock'].add(filename)
    else:
        contents[exp]['taxa'].add(filename)


In [9]:
file_list = []

for exp in contents.items():
    file_data = {}
    file_data['expedition'] = exp[0]
    file_data['taxa'] = format_filepaths_set(exp[1], 'taxa')
    file_data['age_model'] = format_filepaths_set(exp[1], 'age_model')
    file_data['lithology'] = format_filepaths_set(exp[1], 'lithology')
    file_data['hard_rock'] = format_filepaths_set(exp[1], 'hard_rock')

    file_list.append(file_data)

In [10]:
df = pd.DataFrame(file_list)
df.head()

,expedition,taxa,age_model,lithology,hard_rock
0,61,"nannos.csv,p_forams.csv,radiolar.csv,b_forams.csv",ageprof.csv,vistxt.csv,hr_desc.csv
1,95,"diatoms.csv,nannos.csv,radiolar.csv,ostracod.c...",ageprof.csv,vistxt.csv,
2,59,"p_forams.csv,diatoms.csv,nannos.csv,radiolar.c...",ageprof.csv,vistxt.csv,hr_desc.csv
3,92,"nannos.csv,p_forams.csv",ageprof.csv,vistxt.csv,hr_desc.csv
4,66,"nannos.csv,radiolar.csv",ageprof.csv,vistxt.csv,hr_desc.csv


In [11]:
path = OUTPUT_DIR/'tmp'/'noaa_dsdp_grouped_files.csv'
df.to_csv(path, index=False)